In [0]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from scipy.sparse import hstack
import pandas as pd
import numpy as np
import json
import csv
import time 

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link_text = 'https://drive.google.com/open?id=10d3wzsa1g4gJcjH8V53PPr9SQ7ZKp2X9' # The shareable link
fluff, id_1 = link_text.split('=')
print (id_1)

link_star_count = 'https://drive.google.com/open?id=11098un8pO0NLfITzXqUKrdykxjlpqg7I'
fluff, id_2 = link_star_count.split('=')
print (id_2)



    100% |████████████████████████████████| 993kB 1.5MB/s 
10d3wzsa1g4gJcjH8V53PPr9SQ7ZKp2X9
11098un8pO0NLfITzXqUKrdykxjlpqg7I


In [0]:
#importing data from saved tsv files 
pd.set_option('display.max_colwidth', -1)

downloaded = drive.CreateFile({'id':id_1}) 
downloaded.GetContentFile("final_business_star.tsv")  
df_text = pd.read_csv("final_business_star.tsv",delimiter="\t",encoding="utf-8", nrows = 500000) #checkig first 0.5 million rows
                          
downloaded = drive.CreateFile({'id':id_2}) 
downloaded.GetContentFile("final_stars_and_count.tsv")                          
df_star_count = pd.read_csv("final_stars_and_count.tsv",delimiter="\t",encoding="utf-8")

print(df_text.info())

print("Null check stars:",df_star_count["stars"].isnull().sum())

print("df_review_text:", df_text.shape)
print("df_star shape:", df_star_count.shape)

NameError: ignored

In [0]:
df_review_agg = df_text.groupby('business_id')["text"].sum()
df_ready = pd.DataFrame({"business_id":df_review_agg.index,"all_reviews":df_review_agg.values})# "count" : df_star_agg.values })
df_star_ready = pd.DataFrame(df_star_count)

df = pd.merge(df_ready, df_star_ready, on = "business_id")

print(df.info())
print(list(df))  #['business_id', 'all_reviews', 'stars', 'review_count']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18983 entries, 0 to 18982
Data columns (total 4 columns):
all_reviews     18983 non-null object
business_id     18983 non-null object
stars           18983 non-null float64
review_count    18983 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 741.5+ KB
None
['all_reviews', 'business_id', 'stars', 'review_count']


In [0]:
df = df.sort_values("review_count", ascending = False) #sorted data on total number of review a business_id has
# df = df.sample(frac = 1).reset_index(drop=True)
final_df = df.iloc[:750,:]

# considering first x business_id and their data
print(final_df.iloc[:5,1:])
print(final_df.shape)
print(final_df["stars"].mean())

                  business_id  stars  review_count
17414  ujHiaprwCQ5ewziu0Vi9rw  3.5    4227        
12956  faPVqws-x-5k2CQKDNtHxw  4.5    3576        
10141  Wxxvi3LZbHNIDwJ-ZimtnA  4.0    3499        
10311  XXW_OFaYQkkGOGniujZFHg  4.0    3260        
5643   HhVmDybpU7L50Kb5A0jXTg  3.5    3222        
(750, 4)
3.820666666666667


In [0]:
count_list = final_df["review_count"] #creating data of review_count 
count_list = np.array(count_list)[:,None] #converting it to 2D array to pass in normalizer
count_list = normalize(count_list, norm = "l1", axis = 0)

In [0]:
vectorizer = TfidfVectorizer( stop_words = "english", max_features = 1500 , norm = "l1", min_df = 350 , ngram_range = (1,2))
review_list = final_df["all_reviews"]
vector = vectorizer.fit_transform(review_list)
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_)

features_join = hstack((vector,count_list)).A
target = final_df.iloc[:,2]

total_neighbors = len(set(target.values))
print(total_neighbors)

['00', '10', '10 minutes', '100', '11', '12', '13', '14', '15', '15 minutes', '16', '18', '20', '20 minutes', '24', '25', '2nd', '30', '30 minutes', '35', '40', '45', '45 minutes', '50', '60', '75', '90', '95', '99', 'able', 'absolute', 'absolutely', 'absolutely delicious', 'accommodating', 'actual', 'actually', 'add', 'added', 'addition', 'additional', 'admit', 'affordable', 'afternoon', 'ago', 'agree', 'agreed', 'ahead', 'air', 'alcohol', 'allowed', 'alright', 'amazing', 'amazing food', 'amazing service', 'ambiance', 'ambience', 'american', 'annoying', 'answer', 'anymore', 'apart', 'apologized', 'app', 'apparently', 'appetizer', 'appetizers', 'apple', 'appreciate', 'appreciated', 'area', 'areas', 'aren', 'arrive', 'arrived', 'asian', 'ask', 'asked', 'asking', 'ass', 'ate', 'atmosphere', 'atmosphere great', 'attention', 'attentive', 'attitude', 'authentic', 'available', 'average', 'avoid', 'away', 'awesome', 'awful', 'awhile', 'baby', 'bacon', 'bad', 'bag', 'baked', 'balance', 'bar', 

In [0]:
x_train, x_test, y_train, y_test = train_test_split(features_join,target, test_size = 0.33, random_state = 9)
clf = LinearRegression()
classifier = clf.fit(x_train,y_train)
print("Score on training data: ",clf.score(x_train,y_train))
print("Score on testing data: ",clf.score(x_test,y_test))
predict = clf.predict(x_test)
y_test_list= y_test.tolist()
predict_list = predict.tolist()
predict_list = [ round(x,1) for x in predict_list]
print(y_test_list[:5])
print(predict_list[:5])


Score on training data:  1.0
Score on testing data:  0.7154686594146008
[3.0, 4.0, 1.5, 4.5, 4.0]
[3.4, 4.1, 1.9, 4.2, 3.9]


In [0]:
lab_encoder = LabelEncoder()
target_encoded = lab_encoder.fit_transform(target.values)

print( "Shape of features:",features_join.shape)
print( "shape of target:",len(target))

x_train, x_test, y_train, y_test = train_test_split(features_join,target_encoded, test_size = 0.2, random_state = 9)

print("shape of x_train:",x_train.shape)
print("shape of y_train:", y_train.shape)

Shape of features: (750, 1501)
shape of target: 750
shape of x_train: (600, 1501)
shape of y_train: (600,)


In [0]:
ml_models = [ KNeighborsClassifier(n_neighbors = total_neighbors), LogisticRegression(),LinearSVC(), MultinomialNB(alpha = 0.2) ]
name = [ "KNeighborsClassifier", "LinearRegression", "svm___LinearSVC", "MultinomialNB"]
scores = {}
# for index,model in enumerate(ml_models):
for i,model in enumerate(ml_models):
  clf = model
  classifier = clf.fit(x_train,y_train)
  print("\n {}".format(name[i]))
  print("Score on training data: ",clf.score(x_train,y_train))
  print("Score on testing data: ",clf.score(x_test,y_test))
 
  predictions = clf.predict(x_test)

  y_test_list= y_test.tolist()
  predict_list = predictions.tolist()
  print(lab_encoder.inverse_transform(y_test_list[:5]))
  print(lab_encoder.inverse_transform(predict_list[:5]))



 KNeighborsClassifier
Score on training data:  0.5566666666666666
Score on testing data:  0.43333333333333335
[3.  4.  1.5 4.5 4. ]
[3.5 4.  1.5 4.5 4. ]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)



 LinearRegression
Score on training data:  0.4116666666666667
Score on testing data:  0.35333333333333333
[3.  4.  1.5 4.5 4. ]
[4. 4. 4. 4. 4.]

 svm___LinearSVC
Score on training data:  0.4116666666666667
Score on testing data:  0.35333333333333333
[3.  4.  1.5 4.5 4. ]
[4. 4. 4. 4. 4.]

 MultinomialNB
Score on training data:  0.4116666666666667
Score on testing data:  0.35333333333333333
[3.  4.  1.5 4.5 4. ]
[4. 4. 4. 4. 4.]



              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00        16
           5       0.00      0.00      0.00        36
           6       0.35      1.00      0.52        53
           7       0.00      0.00      0.00        29
           8       0.00      0.00      0.00         4

   micro avg       0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
